#### ***Importing pyspark libs***

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

#### ***Creating SparkSession***

In [2]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('df_fifa_world_cup')
    .getOrCreate()
)

#### ***Show User Interface***

In [3]:
spark

#### ***Reading CSV file***

In [4]:
df = spark.read.csv('arquivo/wc2018-players.csv', header=True, inferSchema=True)

#### ***Structure of Dataframe***

In [5]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



#### ***Show command***

In [6]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



#### ***Verifying if exists null fields***

In [7]:
df.toPandas().isna().sum()

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

#### ***Renaming columns***

In [8]:
df = df.withColumnRenamed('Team', 'Selecao')\
       .withColumnRenamed('#', 'Numero')\
       .withColumnRenamed('Pos.', 'Posicao')\
       .withColumnRenamed('FIFA Popular Name', 'Nome_FIFA')\
       .withColumnRenamed('Birth Date', 'Data_Nascimento')\
       .withColumnRenamed('Shirt Name', 'Nome_Camiseta')\
       .withColumnRenamed('Club', 'Time')\
       .withColumnRenamed('Height', 'Altura')\
       .withColumnRenamed('Weight', 'Peso')

df.show(5)

+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|     31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|     21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|     15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|     13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|     24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+
only showing top 5 

#### ***Applying filter in each column to verify if null field exists***

In [9]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Data_Nascimento 0
Nome_Camiseta 0
Time 0
Altura 0
Peso 0


In [10]:
df.select('Selecao', 'Nome_FIFA',).show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [11]:
df.select(col('Selecao').alias('Timaço')).show(5)

+---------+
|   Timaço|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [12]:
df.select(col('Selecao'), col('Nome_FIFA')).show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [13]:
df.select(df['Selecao'], df['Nome_FIFA']).show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [14]:
df.select('Selecao Nome_FIFA Altura'.split()).show(5)

+---------+------------------+------+
|  Selecao|         Nome_FIFA|Altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



#### ***DF filter***

In [15]:
df.filter(df['Selecao'] == 'Brazil').show(10)

+-------+------+-------+-----------------+---------------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+---------------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|     05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|     13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|     04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|     11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|     05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|     12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|    

#### ***Filter DF with 2 conditions (AND / &)***

In [16]:
df.filter((df['Selecao'] == 'Brazil') & (df['Nome_FIFA'] == 'FRED') | (df['Nome_Camiseta'] == 'TAISON')).show(truncate=False)

+-------+------+-------+---------+---------------+-------------+-------------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Data_Nascimento|Nome_Camiseta|Time                     |Altura|Peso|
+-------+------+-------+---------+---------------+-------------+-------------------------+------+----+
|Brazil |18    |MF     |FRED     |05.03.1993     |FRED         |FC Shakhtar Donetsk (UKR)|169   |64  |
|Brazil |21    |FW     |TAISON   |13.01.1988     |TAISON       |FC Shakhtar Donetsk (UKR)|172   |64  |
+-------+------+-------+---------+---------------+-------------+-------------------------+------+----+



In [17]:
df.filter((df['Selecao'] == 'Argentina') & ((df['Altura'] > 190))).show()

+---------+------+-------+--------------+---------------+-------------+-----------------+------+----+
|  Selecao|Numero|Posicao|     Nome_FIFA|Data_Nascimento|Nome_Camiseta|             Time|Altura|Peso|
+---------+------+-------+--------------+---------------+-------------+-----------------+------+----+
|Argentina|     6|     DF|FAZIO Federico|     17.03.1987|        FAZIO|    AS Roma (ITA)|   199|  85|
|Argentina|     1|     GK| GUZMAN Nahuel|     10.02.1986|       GUZMÁN|Tigres UANL (MEX)|   192|  90|
+---------+------+-------+--------------+---------------+-------------+-----------------+------+----+



In [18]:
df.filter((col('Selecao') == "Brazil") & ((col('Altura') > 190) & ((col('Peso') > 91)))).show()

+-------+------+-------+---------+---------------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+---------------+-------------+--------------------+------+----+
| Brazil|    16|     GK|   CASSIO|     06.06.1987|       CASSIO|SC Corinthians (BRA)|   195|  92|
+-------+------+-------+---------+---------------+-------------+--------------------+------+----+



In [19]:
df.filter('Selecao = "Brazil"').filter(col('Numero') > 20).show()

+-------+------+-------+---------+---------------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+---------------+-------------+--------------------+------+----+
| Brazil|    21|     FW|   TAISON|     13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    22|     DF|   FAGNER|     11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    23|     GK|  EDERSON|     17.08.1993|      EDERSON|Manchester City F...|   188|  86|
+-------+------+-------+---------+---------------+-------------+--------------------+------+----+



#### ***Filter DF with 2 conditions (OR / |)***

In [20]:
df.filter((df['Nome_FIFA'] == 'MESSI Lionel') | ((df['Nome_FIFA'] == 'TAGLIAFICO Nicolas'))).show()

+---------+------+-------+------------------+---------------+-------------+------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Data_Nascimento|Nome_Camiseta|              Time|Altura|Peso|
+---------+------+-------+------------------+---------------+-------------+------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|     31.08.1992|   TAGLIAFICO|    AFC Ajax (NED)|   169|  65|
|Argentina|    10|     FW|      MESSI Lionel|     24.06.1987|        MESSI|FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+---------------+-------------+------------------+------+----+



#### ***Filter DF combining & and | (AND / OR)***

In [21]:
df.filter((df['Selecao'] == 'Brazil')).show(23)

+-------+------+-------+-----------------+---------------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+---------------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|     05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|     13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|     04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|     11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|     05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|     12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|    

In [144]:
df.filter((df['Selecao'] == 'Brazil') & ((df['Posicao'] == 'FW') | ((df['Nome_Camiseta'] == 'P. COUTINHO')) | ((df['Time'] == 'SC Corinthians (BRA)')))).show()

+-------+------+-------+-----------------+-------------+--------------------+------+----+----------+
|Selecao|Numero|Posicao|        Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Nascimento|
+-------+------+-------+-----------------+-------------+--------------------+------+----+----------+
| Brazil|    21|     FW|           TAISON|       TAISON|FC Shakhtar Donet...|   172|  64|1988-01-13|
| Brazil|    22|     DF|           FAGNER|       FAGNER|SC Corinthians (BRA)|   168|  67|1989-06-11|
| Brazil|    10|     FW|           NEYMAR|    NEYMAR JR|Paris Saint-Germa...|   175|  68|1992-02-05|
| Brazil|    11|     MF|PHILIPPE COUTINHO|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|1992-06-12|
| Brazil|     7|     FW|    DOUGLAS COSTA|     D. COSTA|   Juventus FC (ITA)|   182|  70|1990-09-14|
| Brazil|     9|     FW|    GABRIEL JESUS|     G. JESUS|Manchester City F...|   175|  73|1997-04-03|
| Brazil|    20|     FW|  ROBERTO FIRMINO|      FIRMINO|  Liverpool FC (ENG)|   181|  76|19

#### ***Creating nem columns using lit function***

In [23]:
df.withColumn('coluna_nova', lit(col('Altura') - col('Peso'))).show(5)

+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+-----------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|coluna_nova|
+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+-----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|     31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|        104|
|Argentina|    22|     MF|    PAVON Cristian|     21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|        104|
|Argentina|    15|     MF|    LANZINI Manuel|     15.02.1993|      LANZINI|West Ham United F...|   167|  66|        101|
|Argentina|    18|     DF|    SALVIO Eduardo|     13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         98|
|Argentina|    10|     FW|      MESSI Lionel|     24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|         98|
+---------+------+-------+------

#### ***Creating conditional column (using substring function)***

In [24]:
df.withColumn('Sub', substring('Selecao', 1, 3)).show(5)

+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|Sub|
+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|     31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina|    22|     MF|    PAVON Cristian|     21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina|    15|     MF|    LANZINI Manuel|     15.02.1993|      LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|     13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina|    10|     FW|      MESSI Lionel|     24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+------+-------+------------------+---------------+-------------+--------------------+

In [25]:
df.show(23)

+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Data_Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+---------------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|     31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|     21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|     15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|     13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|     24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
|Argentina|     4|     DF|  ANSALDI Cristian|     20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|     5| 

In [26]:
df = df.withColumn('Ano', substring('Data_Nascimento', -4,4))

df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Data_Nascimento: string (nullable = true)
 |-- Nome_Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)



In [27]:
df = df.withColumn('Mes', substring('Data_Nascimento', -7, 2))

In [28]:
df = df.withColumn('Dia', substring('Data_Nascimento', -10, 2))

In [29]:
df = df.withColumn('Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia'))

df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Data_Nascimento: string (nullable = true)
 |-- Nome_Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Nascimento: string (nullable = false)



In [30]:
df = df.withColumn('Nascimento', col('Nascimento').cast(DateType()))

df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Data_Nascimento: string (nullable = true)
 |-- Nome_Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Nascimento: date (nullable = true)



In [32]:
df = df.drop('Data_Nascimento', 'Ano', 'Mes', 'Dia')

df.show(5)

+---------+------+-------+------------------+-------------+--------------------+------+----+----------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Nascimento|
+---------+------+-------+------------------+-------------+--------------------+------+----+----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|        PAVÓN|CA Boca Juniors (...|   169|  65|1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|      LANZINI|West Ham United F...|   167|  66|1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|       SALVIO|    SL Benfica (POR)|   167|  69|1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|        MESSI|  FC Barcelona (ESP)|   170|  72|1987-06-24|
+---------+------+-------+------------------+-------------+--------------------+------+----+----------+
only showing top 5 rows



In [116]:
df_total_selecao = df.select(col('Selecao')).distinct().count()

In [130]:
print("Número total de Seleções na Copa do Mundo é igual a", df_total_selecao)

Número total de Seleções na Copa do Mundo é igual a 32


In [149]:
df.filter((df['Selecao'] == 'Belgium') & ((df['Posicao'] == 'FW'))).show(5)

+-------+------+-------+---------------+-------------+--------------------+------+----+----------+
|Selecao|Numero|Posicao|      Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Nascimento|
+-------+------+-------+---------------+-------------+--------------------+------+----+----------+
|Belgium|    14|     FW|  MERTENS Dries|      MERTENS|    SSC Napoli (ITA)|   169|  61|1987-05-06|
|Belgium|    10|     FW|    HAZARD Eden|    E. HAZARD|    Chelsea FC (ENG)|   173|  74|1991-01-07|
|Belgium|    18|     FW|  JANUZAJ Adnan|      JANUZAJ| Real Sociedad (ESP)|   180|  75|1995-02-05|
|Belgium|    21|     FW|BATSHUAYI Michy|    BATSHUAYI|Borussia Dortmund...|   185|  78|1993-10-02|
|Belgium|     9|     FW|  LUKAKU Romelu|    R. LUKAKU|Manchester United...|   190|  94|1993-05-13|
+-------+------+-------+---------------+-------------+--------------------+------+----+----------+

